# 02. Preprocessing

Notebook này sẽ sử dụng các hàm từ `src/data_processing.py` để tiền xử lý dữ liệu.


In [1]:
import sys
sys.path.append('../src')

import numpy as np
from data_processing import (
    load_csv_data,
    get_missing_stats,
    get_numeric_column,
    get_target,
    get_column_by_name,
    process_experience_column,
    get_categorical_stats
)


## 1. Load dữ liệu


In [2]:
# Load dữ liệu train
TRAIN_PATH = '../data/raw/aug_train.csv'
data, header = load_csv_data(TRAIN_PATH)

print(f"Kích thước dữ liệu: {data.shape}")
print(f"Số mẫu: {data.shape[0]}, Số features: {data.shape[1]}")
print(f"Các cột: {header}")


Kích thước dữ liệu: (19158, 14)
Số mẫu: 19158, Số features: 14
Các cột: ['enrollee_id', 'city', 'city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'target']


## 2. Xử lý Missing Values


### 2.1. Xử lý các cột Numeric


In [4]:
# Xử lý các cột numeric
cdi = get_numeric_column(data, header, 'city_development_index')
training_hours = get_numeric_column(data, header, 'training_hours')

# experience: có missing và giá trị đặc biệt (>20, <1, never)
experience = get_numeric_column(data, header, 'experience', process_experience=True)

print("Các cột numeric:")
print(f"city_development_index: mean={np.nanmean(cdi):.4f}, std={np.nanstd(cdi):.4f}, missing={np.sum(np.isnan(cdi))}")
print(f"training_hours: mean={np.nanmean(training_hours):.2f}, std={np.nanstd(training_hours):.2f}, missing={np.sum(np.isnan(training_hours))}")
print(f"experience: mean={np.nanmean(experience):.2f}, std={np.nanstd(experience):.2f}, missing={np.sum(np.isnan(experience))}")

# Impute missing values cho experience bằng median
experience_median = np.nanmedian(experience)
experience_filled = experience.copy()
experience_filled[np.isnan(experience_filled)] = experience_median
print(f"\nExperience sau khi impute (median={experience_median:.2f}): missing={np.sum(np.isnan(experience_filled))}")


Các cột numeric:
city_development_index: mean=0.8288, std=0.1234, missing=0
training_hours: mean=65.37, std=60.06, missing=0
experience: mean=10.11, std=6.76, missing=65

Experience sau khi impute (median=9.00): missing=0


### 2.2. Xử lý các cột Categorical - One-hot Encoding


In [ ]:
def one_hot_encode(data, header, colname, fill_missing='Missing'):
    """
    One-hot encode cho một cột categorical
    """
    col = get_column_by_name(data, header, colname)
    col_filled = col.copy()
    
    # Thay missing bằng giá trị fill_missing
    col_filled[col_filled == ''] = fill_missing
    
    # Lấy unique categories
    categories = np.unique(col_filled)
    
    # Tạo one-hot encoding
    n_samples = len(col_filled)
    n_categories = len(categories)
    encoded = np.zeros((n_samples, n_categories), dtype=int)
    
    for i, category in enumerate(categories):
        encoded[col_filled == category, i] = 1
    
    return encoded, categories.tolist()


# Danh sách các cột categorical cần encode.
# Bỏ các feature ,'gender', 'company_type', 'company_size','major_discipline'. Vì có sự áp đảo của 1 dữ liệu so với các dữ liệu khác

categorical_cols = [
    'relevent_experience', 'enrolled_university', 
    'education_level',
    'last_new_job'
]

# One-hot encode từng cột
encoded_features = {}
all_categories = {}

for colname in categorical_cols:
    encoded, categories = one_hot_encode(data, header, colname)
    encoded_features[colname] = encoded
    all_categories[colname] = categories
    print(f"{colname:25s}: {encoded.shape[1]} categories, missing được thay bằng 'Missing'")


relevent_experience      : 2 categories, missing được thay bằng 'Missing'
enrolled_university      : 4 categories, missing được thay bằng 'Missing'
education_level          : 6 categories, missing được thay bằng 'Missing'
last_new_job             : 7 categories, missing được thay bằng 'Missing'


### 2.3. Xử lý cột city - Label Encoding (vì có quá nhiều city)


In [6]:
def label_encode(data, header, colname):
    """
    Label encode cho một cột (chuyển string thành số)
    """
    col = get_column_by_name(data, header, colname)
    unique_vals = np.unique(col)
    
    # Tạo mapping
    label_map = {val: i for i, val in enumerate(unique_vals)}
    
    # Encode
    encoded = np.array([label_map[val] for val in col], dtype=int)
    
    return encoded, label_map


# Label encode cho city (vì có quá nhiều city để one-hot)
city_encoded, city_label_map = label_encode(data, header, 'city')
print(f"city: {len(city_label_map)} unique values, encoded thành label 0-{len(city_label_map)-1}")


city: 123 unique values, encoded thành label 0-122


## 3. Kết hợp tất cả features


In [7]:
# Tạo danh sách các feature arrays
feature_list = []

# 1. Numeric features (base features)
feature_list.append(cdi.reshape(-1, 1))  # city_development_index
feature_list.append(training_hours.reshape(-1, 1))  # training_hours
feature_list.append(experience_filled.reshape(-1, 1))  # experience (đã impute)
feature_list.append(city_encoded.reshape(-1, 1))  # city (label encoded)

print("Base numeric features: CDI, Training_hours, Experience, City")

# 2. Feature Engineering: Interaction và Polynomial Features
# Dựa trên phân tích tương quan:
# - CDI và Target: -0.34 (tương quan mạnh)

# 2.1. Interaction features (tương quan giữa các biến)
print("\nTạo Interaction Features...")
# CDI * Experience (vì có tương quan 0.33)
cdi_exp_interaction = (cdi * experience_filled).reshape(-1, 1)
feature_list.append(cdi_exp_interaction)
print("  - CDI * Experience")

# 2.2. Polynomial features (bậc 2 cho các biến quan trọng)
print("\nTạo Polynomial Features (bậc 2)...")
# CDI^2 (vì CDI tương quan mạnh với target)
cdi_squared = (cdi ** 2).reshape(-1, 1)
feature_list.append(cdi_squared)
print("  - CDI^2")

# Experience^2
exp_squared = (experience_filled ** 2).reshape(-1, 1)
feature_list.append(exp_squared)
print("  - Experience^2")

# 2.3. Ratio features (tỷ lệ giữa các biến)
print("\nTạo Ratio Features...")
# Training_hours / (Experience + 1) - để tránh chia cho 0
training_per_exp = (training_hours / (experience_filled + 1)).reshape(-1, 1)
feature_list.append(training_per_exp)
print("  - Training_hours / (Experience + 1)")

# CDI / (Experience + 1)
cdi_per_exp = (cdi / (experience_filled + 1)).reshape(-1, 1)
feature_list.append(cdi_per_exp)
print("  - CDI / (Experience + 1)")

# 2.4. Log transform cho các biến có phân phối lệch
print("\nTạo Log Transform Features...")
# Log(Experience + 1)
exp_log = np.log1p(experience_filled).reshape(-1, 1)
feature_list.append(exp_log)
print("  - Log(Experience + 1)")

# Log(Training_hours + 1)
training_log = np.log1p(training_hours).reshape(-1, 1)
feature_list.append(training_log)
print("  - Log(Training_hours + 1)")

# 3. Categorical features (one-hot encoded)
print("\nThêm Categorical Features (one-hot)...")
for colname in categorical_cols:
    feature_list.append(encoded_features[colname])
    print(f"  - {colname} ({encoded_features[colname].shape[1]} categories)")

# Kết hợp tất cả features
X = np.hstack(feature_list)

print(f"\n" + "=" * 60)
print(f"Shape của X: {X.shape}")
print(f"Số features: {X.shape[1]} (base: 4, engineered: {X.shape[1] - 4 - sum([encoded_features[col].shape[1] for col in categorical_cols])}, categorical: {sum([encoded_features[col].shape[1] for col in categorical_cols])})")
print("=" * 60)

# Get target
y = get_target(data, header)
print(f"\nShape của y: {y.shape}")
print(f"Phân phối target: {np.bincount(y)}")


Base numeric features: CDI, Training_hours, Experience, City

Tạo Interaction Features...
  - CDI * Experience

Tạo Polynomial Features (bậc 2)...
  - CDI^2
  - Experience^2

Tạo Ratio Features...
  - Training_hours / (Experience + 1)
  - CDI / (Experience + 1)

Tạo Log Transform Features...
  - Log(Experience + 1)
  - Log(Training_hours + 1)

Thêm Categorical Features (one-hot)...
  - relevent_experience (2 categories)
  - enrolled_university (4 categories)
  - education_level (6 categories)
  - last_new_job (7 categories)

Shape của X: (19158, 30)
Số features: 30 (base: 4, engineered: 7, categorical: 19)

Shape của y: (19158,)
Phân phối target: [14381  4777]


## 4. Feature Engineering - Tạo thêm features từ tương quan

Đã thêm các interaction và polynomial features dựa trên phân tích tương quan:
- CDI và Experience có tương quan 0.33 → Tạo CDI * Experience
- CDI và Target có tương quan -0.34 → Tạo CDI^2
- Các ratio và log transform features

## 5. Normalization - Chuẩn hóa features


In [8]:
def normalize_features(X, method='standard'):
    if method == 'standard':
        mean = np.mean(X, axis=0)
        std = np.std(X, axis=0)
        # Tránh chia cho 0
        std[std == 0] = 1
        X_norm = (X - mean) / std
        return X_norm, mean, std, None, None
    
    elif method == 'minmax':
        min_vals = np.min(X, axis=0)
        max_vals = np.max(X, axis=0)
        # Tránh chia cho 0
        range_vals = max_vals - min_vals
        range_vals[range_vals == 0] = 1
        X_norm = (X - min_vals) / range_vals
        return X_norm, None, None, min_vals, max_vals


# Chuẩn hóa bằng StandardScaler (z-score normalization)
X_normalized, X_mean, X_std, _, _ = normalize_features(X, method='standard')

print(f"X_normalized shape: {X_normalized.shape}")
print(f"Mean sau normalization: {np.mean(X_normalized, axis=0)[:5]}")  # Hiển thị 5 features đầu
print(f"Std sau normalization: {np.std(X_normalized, axis=0)[:5]}")  # Hiển thị 5 features đầu


X_normalized shape: (19158, 30)
Mean sau normalization: [-6.61351404e-14 -9.02874764e-17 -3.07359896e-16  3.29381231e-16
 -4.76675455e-14]
Std sau normalization: [1. 1. 1. 1. 1.]


## 6. Thêm bias term (intercept)


In [9]:
# Thêm cột bias (toàn bộ là 1) vào đầu feature matrix
n_samples = X_normalized.shape[0]
bias = np.ones((n_samples, 1))
X_final = np.hstack([bias, X_normalized])

print(f"X_final shape (sau khi thêm bias): {X_final.shape}")
print(f"Features: {X_final.shape[1]} (1 bias + {X_normalized.shape[1]} features)")


X_final shape (sau khi thêm bias): (19158, 31)
Features: 31 (1 bias + 30 features)


## 7. Lưu dữ liệu đã preprocess


In [10]:
# Lưu dữ liệu đã preprocess với stratified train/test split
# Thiết lập test_size
TEST_SIZE = 0.2
RANDOM_SEED = 42

# Stratified split (tỷ lệ giữ nguyên giữa các class)
class0_idx = np.where(y == 0)[0]
class1_idx = np.where(y == 1)[0]

rng = np.random.default_rng(RANDOM_SEED)
rng.shuffle(class0_idx)
rng.shuffle(class1_idx)

n_class0_test = int(len(class0_idx) * TEST_SIZE)
n_class1_test = int(len(class1_idx) * TEST_SIZE)

test_idx = np.concatenate([class0_idx[:n_class0_test], class1_idx[:n_class1_test]])
train_idx = np.concatenate([class0_idx[n_class0_test:], class1_idx[n_class1_test:]])

# Shuffle final indices to avoid ordering by class
rng.shuffle(train_idx)
rng.shuffle(test_idx)

X_train_split = X_final[train_idx]
y_train_split = y[train_idx]
X_test_split = X_final[test_idx]
y_test_split = y[test_idx]

# Lưu train/test sau split
np.save('../data/processed/X_train.npy', X_train_split)
np.save('../data/processed/y_train.npy', y_train_split)
np.save('../data/processed/X_test.npy', X_test_split)
np.save('../data/processed/y_test.npy', y_test_split)

# Lưu thêm các thông tin cần thiết cho preprocessing
import pickle
preprocessing_info = {
    'experience_median': experience_median,
    'city_label_map': city_label_map,
    'all_categories': all_categories,
    'categorical_cols': categorical_cols,
    'test_size': TEST_SIZE,
    'random_seed': RANDOM_SEED
}
with open('../data/processed/preprocessing_info.pkl', 'wb') as f:
    pickle.dump(preprocessing_info, f)

print("=" * 60)
print("ĐÃ LƯU DỮ LIỆU SAU STRATIFIED SPLIT:")
print("=" * 60)
print("\nTrain/Test split (từ aug_train.csv ban đầu):")
print(f"  - Test size: {TEST_SIZE*100:.1f}% (stratified)")
print(f"  - X_train.npy: {X_train_split.shape}")
print(f"  - y_train.npy: {y_train_split.shape} | Phân phối: {np.bincount(y_train_split)}")
print(f"  - X_test.npy : {X_test_split.shape}")
print(f"  - y_test.npy : {y_test_split.shape} | Phân phối: {np.bincount(y_test_split)}")

print("\nPreprocessing info:")
print("  - preprocessing_info.pkl")

print(f"\n" + "=" * 60)
print("Tổng kết cuối cùng:")
print(f"  - Tổng mẫu ban đầu: {X_final.shape[0]:,}")
print(f"  - Train: {X_train_split.shape[0]:,} mẫu")
print(f"  - Test : {X_test_split.shape[0]:,} mẫu")
print(f"  - Features mỗi mẫu: {X_final.shape[1]}")
print("=" * 60)

ĐÃ LƯU DỮ LIỆU SAU STRATIFIED SPLIT:

Train/Test split (từ aug_train.csv ban đầu):
  - Test size: 20.0% (stratified)
  - X_train.npy: (15327, 31)
  - y_train.npy: (15327,) | Phân phối: [11505  3822]
  - X_test.npy : (3831, 31)
  - y_test.npy : (3831,) | Phân phối: [2876  955]

Preprocessing info:
  - preprocessing_info.pkl

Tổng kết cuối cùng:
  - Tổng mẫu ban đầu: 19,158
  - Train: 15,327 mẫu
  - Test : 3,831 mẫu
  - Features mỗi mẫu: 31
